In [1]:
import face_recognition
import cv2
import numpy as np
import glob
from winsound import Beep

In [2]:
video_capture = cv2.VideoCapture(0)

my_imgs = glob.glob('faces/*.jpg')

# print(my_imgs)

known_face_encodings = []
known_face_names = []
face_locations = []
face_encodings = []
face_names = []
imgs = []
process_this_frame = True

In [3]:
for i in my_imgs:
    imgs.append(face_recognition.load_image_file(i))
    known_face_names.append(my_imgs[my_imgs.index(i)][6:-4])

for i in imgs:
    known_face_encodings.append(face_recognition.face_encodings(i)[0])

In [4]:
while True:
    ret, frame = video_capture.read()

    if process_this_frame:
        small_frame = cv2.resize(frame, (0, 0), fx=0.25, fy=0.25)

        rgb_small_frame = cv2.cvtColor(small_frame, cv2.COLOR_BGR2RGB)

        face_locations = face_recognition.face_locations(rgb_small_frame)
        face_encodings = face_recognition.face_encodings(rgb_small_frame, face_locations)

        face_names = []
        for face_encoding in face_encodings:
            matches = face_recognition.compare_faces(known_face_encodings, face_encoding)
            name = "Unknown"


            face_distances = face_recognition.face_distance(known_face_encodings, face_encoding)
            best_match_index = np.argmin(face_distances)
            if matches[best_match_index]:
                name = known_face_names[best_match_index]

            face_names.append(name)


    process_this_frame = not process_this_frame

    # Displaying the results
    for (top, right, bottom, left), name in zip(face_locations, face_names):
        top *= 4
        right *= 4
        bottom *= 4
        left *= 4

        cv2.rectangle(frame, (left, top), (right, bottom), (0, 0, 255), 2)
        cv2.rectangle(frame, (left, bottom - 35), (right, bottom), (0, 0, 255), cv2.FILLED)
        
        font = cv2.FONT_HERSHEY_DUPLEX
        cv2.putText(frame, name, (left + 6, bottom - 6), font, 1.0, (255, 255, 255), 1)


    cv2.imshow('Video', frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

video_capture.release()
cv2.destroyAllWindows()